# Data Preprocessing for Humpback Whale YouTube Video Dataset 

TO DO: 
- configure the cells above to convert urls-->mp4 files and upload videos onto google folder using drive API
    - extract metadata such as duration and title. Compile this info (mp4 name, title, duration) into a pandas dataframe and save as a csv file
- visualize duration of videos (rel vs non rel EDA continuation) --> use this info to determine at what x frame we want to extract the frame

Links

- https://keras.io/examples/vision/video_classification/
- https://www.twilio.com/blog/media-file-storage-python-flask-amazon-s3-buckets
- https://aws.amazon.com/blogs/media/amazon-s3-glacier-for-infinite-video-storage-with-ross/

### Step 1: Download YouTube videos from URLs onto Local Machine (or Google Drive)
- https://towardsdatascience.com/the-easiest-way-to-download-youtube-videos-using-python-2640958318ab

In [2]:
# !pip install --upgrade pytube
# !pip install --upgrade google-api-python-client

In [ ]:
# !pip uninstall pytube
# !pip install --upgrade pytube

In [3]:
#to reflect changes made in modules
%load_ext autoreload
%autoreload 2

In [4]:
from pytube import YouTube

#misc
import os
import shutil
import math
import datetime
import numpy as np
import time

#plots
import matplotlib.pyplot as plt
%matplotlib inline

#pandas
import pandas as pd

#image operation
import cv2

# #to upload files (local --> google drive)
# from google.colab import files

In [5]:
# provide access to your google drive
# you can see the files in your google drive under the 'content' folder and inside 
# gdrive' folder
# tutorial: https://neptune.ai/blog/google-colab-dealing-with-files
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# creating a google drive folder to upload our files to
# folder_name = 'humpback_whale_yt_videos_capstone' 
# folder_name = 'humpback_whale_yt_videos_capstone_additional_irrelevant_videos' #Folder ID: 1RicLj-NgjnZDXsVGAqqokdsu1OlQb2vE 

# Get YouTube Video and Upload mp4 File to Google Drive

### Testing - Downloading a video

In [ ]:
# To address HTTPError 410:Gone with PyTube
# !pip install --upgrade pytube

In [3]:
#create instance of youtube class 
video = YouTube("https://www.youtube.com/watch?v=dE1P4zDhhqw")#YouTube("https://www.youtube.com/watch?v=oSGhnD5kmAg")

In [4]:
#print a summary of the selected video
print('Summary')
print(f'Title: {video.title}')
print(f'Duration: {video.length / 60:.2f} minutes')
print(f'Duration unmod: {video.length}') #video.length is in seconds
print(f'Rating: {video.rating}')
print(f'# of views: {video.views}')

Summary
Title: Captain America "Avengers Assemble" Scene - Portal Scene - Avengers : Endgame (2019) Scene
Duration: 4.92 minutes
Duration unmod: 295
Rating: None
# of views: 15357461


In [5]:
# display only the streams with the selected file format (mp4)
for stream in video.streams.filter(file_extension="mp4").order_by('resolution'):
    print(stream)

<Stream: itag="160" mime_type="video/mp4" res="144p" fps="30fps" vcodec="avc1.4d400c" progressive="False" type="video">
<Stream: itag="394" mime_type="video/mp4" res="144p" fps="30fps" vcodec="av01.0.00M.08" progressive="False" type="video">
<Stream: itag="133" mime_type="video/mp4" res="240p" fps="30fps" vcodec="avc1.4d4015" progressive="False" type="video">
<Stream: itag="395" mime_type="video/mp4" res="240p" fps="30fps" vcodec="av01.0.00M.08" progressive="False" type="video">
<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">
<Stream: itag="134" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.4d401e" progressive="False" type="video">
<Stream: itag="396" mime_type="video/mp4" res="360p" fps="30fps" vcodec="av01.0.01M.08" progressive="False" type="video">
<Stream: itag="135" mime_type="video/mp4" res="480p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">
<Stream: itag="39

# Uploading a pytube video straight to google drive without downloading locally

In [9]:
#get highest res version of video
# highest_res_video = video.streams.filter(file_extension="mp4").order_by('resolution').last()

In [ ]:
# directory = '/content/gdrive/MyDrive/' + folder_name
# for ii in range(0,1):
#   highest_res_video.download(output_path = '/content', filename= f"video_{ii:04d}.mp4")

# Checking how many videos are unavailable in our dataset

In [ ]:
#upload 'humpback whales.csv' to content/sample_data folder in google colab
#note: this is temporary, so will have to reupload whenever we rerun this notebook
uploaded = files.upload() 

Saving humpback whales.csv to humpback whales.csv


In [ ]:
hw_df = pd.read_csv('humpback whales.csv')
hw_df.head()

,_id,channelId,description.eng,description.original,dislikeCount,duration,encounter.dates,encounter.locationIDs,fileDetails,likeCount,newLocation,publishedAt,recordingDetails.date,recordingDetails.location,regionRestriction,relevant,tags.eng,tags.original,title.eng,title.original,uploadedAt,url,user_country,videoID,viewCount,wild
0,8j1HWD8KuMM,NaN,NaN,Tom Mustill - Humpback Whales - A Detective St...,0.0,NaN,[],[],NaN,1.0,0,2019-05-05T21:17:14.000Z,NaN,NaN,NaN,True,[],"[""auto-upload""]",NaN,Tom Mustill - Humpback Whales - A Detective St...,NaN,https://youtu.be/8j1HWD8KuMM,NaN,8j1HWD8KuMM,170,true
1,tW_T8Z6ebmg,NaN,NaN,"Humpback whale calf suckling its mother, first...",3.0,NaN,[],[],NaN,166.0,0,2018-10-15T08:32:31.000Z,NaN,NaN,NaN,True,[],"[""humpback whale"",""suckling behaviour"",""Madaga...",NaN,Suckling Humpback whale,NaN,https://youtu.be/tW_T8Z6ebmg,NaN,tW_T8Z6ebmg,36383,true
2,oSGhnD5kmAg,NaN,NaN,To use this video in a commercial player or br...,1.0,NaN,[],[],NaN,16.0,0,2019-03-07T09:39:33.000Z,NaN,NaN,NaN,True,[],"[""storyful"",""news"",""viral"",""Humpback Whale"",""W...",NaN,Humpback Whales Lunge Feed Among Pod of Dolphi...,NaN,https://youtu.be/oSGhnD5kmAg,NaN,oSGhnD5kmAg,1016,true
3,6eWwFNq4i_c,NaN,NaN,The best moments of the 2017 humpback whale se...,0.0,NaN,[],[],NaN,15.0,0,2017-09-29T06:42:30.000Z,NaN,NaN,NaN,True,[],"[""Whales"",""Swim with whales"",""Wildlife"",""Snork...",NaN,Humpback whales 2017 - Best of by Moorea Ocean...,NaN,https://youtu.be/6eWwFNq4i_c,NaN,6eWwFNq4i_c,2088,true
4,RMaMmynj76s,NaN,NaN,A large group of Humpback Whales lunge feeding...,87.0,NaN,[],[],NaN,1286.0,0,2015-09-17T15:29:19.000Z,NaN,NaN,NaN,True,[],"[""Humpback Whale (Organism Classification)"",""s...",NaN,Amazing! Humpback Whales Lunge Feeding up clos...,NaN,https://youtu.be/RMaMmynj76s,NaN,RMaMmynj76s,377651,true


In [ ]:
#after 156 videos, ratelimiter is blocked and marks all following videos as unavailable 
#might want to divide into batches of 100 videos at a time
count_unavailable = 0
count_avail = 0
urls_unavail = []

start = 0
end = 100
num_batches = int(len(hw_df)/100) + 1

# for i in range(0, num_batches):
for i in range(0, 1):
  print('Batch {}: Start= {} End={}\n'.format(i, start, end))

  for index, row in hw_df[start:end].iterrows():
    try:
      video = YouTube(row['url'])
      video.check_availability() 
      count_avail += 1
    except:
      #print(index, row['url'])
      urls_unavail.append(row['url'])
      count_unavailable += 1
  start += 100
  end += 100

print('Num. Unavailable: ', count_unavailable)
print('Num. Available: ', count_avail)
print('Total: {}\n'.format(len(hw_df)))

Batch 0: Start= 0 End=100

Num. Unavailable:  100
Num. Available:  0
Total:  1585


In [ ]:
# urls_unavail

# Initial Data Download: Create a sub df with ~150 videos total --> 100 rel, 50 irrel

- get subset of truly relevant videos (100)
- get subset of irrelevant videos (50)
- concatenate the two dataframes + create sub_hw_df (reset index + save to csv)
- run the pytube classifier for downloading the videos

In [ ]:
#get subset of 100 truly relevant video
sub_hw_df_rel = hw_df[(hw_df['relevant'] == True)][0:100]
print(len(sub_hw_df_rel))
sub_hw_df_rel.to_csv('/content/sub_hw_df_rel.csv')
sub_hw_df_rel.head()

100


,_id,channelId,description.eng,description.original,dislikeCount,duration,encounter.dates,encounter.locationIDs,fileDetails,likeCount,newLocation,publishedAt,recordingDetails.date,recordingDetails.location,regionRestriction,relevant,tags.eng,tags.original,title.eng,title.original,uploadedAt,url,user_country,videoID,viewCount,wild
0,8j1HWD8KuMM,NaN,NaN,Tom Mustill - Humpback Whales - A Detective St...,0.0,NaN,[],[],NaN,1.0,0,2019-05-05T21:17:14.000Z,NaN,NaN,NaN,True,[],"[""auto-upload""]",NaN,Tom Mustill - Humpback Whales - A Detective St...,NaN,https://youtu.be/8j1HWD8KuMM,NaN,8j1HWD8KuMM,170,true
1,tW_T8Z6ebmg,NaN,NaN,"Humpback whale calf suckling its mother, first...",3.0,NaN,[],[],NaN,166.0,0,2018-10-15T08:32:31.000Z,NaN,NaN,NaN,True,[],"[""humpback whale"",""suckling behaviour"",""Madaga...",NaN,Suckling Humpback whale,NaN,https://youtu.be/tW_T8Z6ebmg,NaN,tW_T8Z6ebmg,36383,true
2,oSGhnD5kmAg,NaN,NaN,To use this video in a commercial player or br...,1.0,NaN,[],[],NaN,16.0,0,2019-03-07T09:39:33.000Z,NaN,NaN,NaN,True,[],"[""storyful"",""news"",""viral"",""Humpback Whale"",""W...",NaN,Humpback Whales Lunge Feed Among Pod of Dolphi...,NaN,https://youtu.be/oSGhnD5kmAg,NaN,oSGhnD5kmAg,1016,true
3,6eWwFNq4i_c,NaN,NaN,The best moments of the 2017 humpback whale se...,0.0,NaN,[],[],NaN,15.0,0,2017-09-29T06:42:30.000Z,NaN,NaN,NaN,True,[],"[""Whales"",""Swim with whales"",""Wildlife"",""Snork...",NaN,Humpback whales 2017 - Best of by Moorea Ocean...,NaN,https://youtu.be/6eWwFNq4i_c,NaN,6eWwFNq4i_c,2088,true
4,RMaMmynj76s,NaN,NaN,A large group of Humpback Whales lunge feeding...,87.0,NaN,[],[],NaN,1286.0,0,2015-09-17T15:29:19.000Z,NaN,NaN,NaN,True,[],"[""Humpback Whale (Organism Classification)"",""s...",NaN,Amazing! Humpback Whales Lunge Feeding up clos...,NaN,https://youtu.be/RMaMmynj76s,NaN,RMaMmynj76s,377651,true


In [ ]:
#get subset of 100 irrelevant videos 
sub_hw_df_irrel = hw_df[(hw_df['relevant'] == False)][50:100]
print(len(sub_hw_df_irrel))
sub_hw_df_irrel.to_csv('/content/sub_hw_df_irrel.csv')
sub_hw_df_irrel.head()

50


,_id,channelId,description.eng,description.original,dislikeCount,duration,encounter.dates,encounter.locationIDs,fileDetails,likeCount,newLocation,publishedAt,recordingDetails.date,recordingDetails.location,regionRestriction,relevant,tags.eng,tags.original,title.eng,title.original,uploadedAt,url,user_country,videoID,viewCount,wild
467,HB-zqdequrs,NaN,NaN,This video is a part of my travel blog - 'Note...,0.0,NaN,[],[],NaN,3.0,NaN,2017-02-25T06:24:32.000Z,NaN,NaN,NaN,False,[],"[""humpback whales"",""sdm diving"",""eli martinez""...",NaN,Humpback Whale Expedition - We arrive on Salt Cay,NaN,https://youtu.be/HB-zqdequrs,NaN,HB-zqdequrs,257,0
468,hsPsulhkZSU,NaN,NaN,Sea kayaking with humpback whales - Bartlett C...,0.0,NaN,[],[],NaN,3.0,NaN,2014-07-22T22:11:31.000Z,NaN,NaN,NaN,False,[],"[""Humpback Whale (Organism Classification)"",""G...",NaN,Sea kayaking with humpback whales - Bartlett C...,NaN,https://youtu.be/hsPsulhkZSU,NaN,hsPsulhkZSU,1127,0
472,7bJApapF7QA,NaN,NaN,OCT. 2 2018 ;; https://www.gofundme.com/sdc76s...,0.0,NaN,[],[],NaN,58.0,NaN,2018-10-02T16:12:31.000Z,NaN,NaN,NaN,False,[],"[""FUKUSHIMA NEWS; whales"",""dead tide pools"",""p...",NaN,Fukushima news; PACIFIC Humpback Whales in ACU...,NaN,https://youtu.be/7bJApapF7QA,NaN,7bJApapF7QA,577,0
473,vaCJLjfIYUY,NaN,NaN,During the 1970's and 1980's my 40 year long f...,0.0,NaN,[],[],NaN,7.0,NaN,2018-06-18T14:55:15.000Z,NaN,NaN,NaN,False,[],"[""Whales"",""Research"",""nature""]",NaN,Judy and her Humpback Whale Research,NaN,https://youtu.be/vaCJLjfIYUY,NaN,vaCJLjfIYUY,82,0
475,annlmtwKcQU,NaN,NaN,The humpback whale (Megaptera novaeangliae) is...,3.0,NaN,[],[],NaN,1.0,NaN,2013-04-02T09:15:27.000Z,NaN,NaN,NaN,False,[],"[""pravda"",""pravda.ru"",""russia"",""moscow"",""humpb...",NaN,Humpback whale: Acrobatic animal,NaN,https://youtu.be/annlmtwKcQU,NaN,annlmtwKcQU,1135,0


Looking through a couple of the irrelevant videos, there are some video that I would actually consider relevant (like news videos on humpback whales, and informational videos). I think it would be a good idea to revisit this dataset (irrelevant only) and double check them + update their relevant status as necessary. For now, we will move on though. 

In [ ]:
#concatenate the sub rel & irrel dataframes and ignore the index
sub_hw_df = pd.concat([sub_hw_df_rel, sub_hw_df_irrel], ignore_index=True)
sub_hw_df.to_csv('/content/sub_hw_df.csv')
print(len(sub_hw_df))
sub_hw_df.head()

150


,_id,channelId,description.eng,description.original,dislikeCount,duration,encounter.dates,encounter.locationIDs,fileDetails,likeCount,newLocation,publishedAt,recordingDetails.date,recordingDetails.location,regionRestriction,relevant,tags.eng,tags.original,title.eng,title.original,uploadedAt,url,user_country,videoID,viewCount,wild
0,8j1HWD8KuMM,NaN,NaN,Tom Mustill - Humpback Whales - A Detective St...,0.0,NaN,[],[],NaN,1.0,0,2019-05-05T21:17:14.000Z,NaN,NaN,NaN,True,[],"[""auto-upload""]",NaN,Tom Mustill - Humpback Whales - A Detective St...,NaN,https://youtu.be/8j1HWD8KuMM,NaN,8j1HWD8KuMM,170,true
1,tW_T8Z6ebmg,NaN,NaN,"Humpback whale calf suckling its mother, first...",3.0,NaN,[],[],NaN,166.0,0,2018-10-15T08:32:31.000Z,NaN,NaN,NaN,True,[],"[""humpback whale"",""suckling behaviour"",""Madaga...",NaN,Suckling Humpback whale,NaN,https://youtu.be/tW_T8Z6ebmg,NaN,tW_T8Z6ebmg,36383,true
2,oSGhnD5kmAg,NaN,NaN,To use this video in a commercial player or br...,1.0,NaN,[],[],NaN,16.0,0,2019-03-07T09:39:33.000Z,NaN,NaN,NaN,True,[],"[""storyful"",""news"",""viral"",""Humpback Whale"",""W...",NaN,Humpback Whales Lunge Feed Among Pod of Dolphi...,NaN,https://youtu.be/oSGhnD5kmAg,NaN,oSGhnD5kmAg,1016,true
3,6eWwFNq4i_c,NaN,NaN,The best moments of the 2017 humpback whale se...,0.0,NaN,[],[],NaN,15.0,0,2017-09-29T06:42:30.000Z,NaN,NaN,NaN,True,[],"[""Whales"",""Swim with whales"",""Wildlife"",""Snork...",NaN,Humpback whales 2017 - Best of by Moorea Ocean...,NaN,https://youtu.be/6eWwFNq4i_c,NaN,6eWwFNq4i_c,2088,true
4,RMaMmynj76s,NaN,NaN,A large group of Humpback Whales lunge feeding...,87.0,NaN,[],[],NaN,1286.0,0,2015-09-17T15:29:19.000Z,NaN,NaN,NaN,True,[],"[""Humpback Whale (Organism Classification)"",""s...",NaN,Amazing! Humpback Whales Lunge Feeding up clos...,NaN,https://youtu.be/RMaMmynj76s,NaN,RMaMmynj76s,377651,true


In [ ]:
uploaded = files.upload()

# Downloading multiple videos from our dataset 

In [9]:
# uploaded = files.upload() #only run if in google drive

In [54]:
csv_name = 'new_videos_to_add.csv' 
#csv_name = 'irrelevant_videos_to_add.csv' 
#csv_name = 'sub_hw_df.csv'

In [55]:
# sub_hw_df = pd.read_csv(csv_name)
sub_hw_df = pd.read_csv(csv_name, index_col = 'Unnamed: 0')
sub_hw_df.head()

,title,url,relevant,occurence_intervals,double_checked,rand_sampled_intervals,double_checke
0,Humpback whale breach 7-10-14,https://youtu.be/I82N9I_SVmM,True,"[('0:30', '0:48')]",True,NaN,NaN
1,Whales-n-Sails flipper slapping humpback whale...,https://youtu.be/bE6FxjPbYcc,True,"[('0:36', '0:50')]",True,NaN,NaN
2,Whales-n-Sails humpback whales July 3/10,https://youtu.be/DIesiEj4XEw,True,"[('0:10', '0:17')]",True,NaN,NaN
3,Female Humpback Whale &quot;Sailing&quot; (Maui),https://youtu.be/CnHl9Zr_07Y,True,"[('0:30', '0:45')]",True,NaN,NaN
4,"2015 Sardine Run 10, Humpback Whale",https://youtu.be/qjg8n42JpUk,True,"[('0:00', '0:15')]",True,NaN,NaN


In [9]:
## directory = '/content/gdrive/MyDrive/humpback_whale_yt_videos_capstone'
## directory = '/content/gdrive/MyDrive/humpback_whale_yt_videos_capstone_2' #videos renamed in new format

# directory =  'additional_irrelevant_videos/' #'/content/gdrive/MyDrive/' + folder_name
directory = 'new_videos_to_add/'

video_dict_list = []

print('working in directory: ', directory)

working in directory:  new_videos_to_add/


In [56]:
len(video_dict_list)

160

In [68]:
#download x videos at a time
start = 198
end = start + 20
print('start: {}, end:{}'.format(start, end))

start: 198, end:218


In [69]:
#download the videos from our sub_hw_df into our directory
for ii, row in sub_hw_df[start:end+1].iterrows():
  # print(row['url'])
  try:
    #get video item, may contain multiple resolution version
    #we also check for availability, as some videos may be no longer available
    video = YouTube(row['url'])
    video.check_availability() 

    #get + download the actual video mp4 version we want to store/keep
    # num = row['filtering_index']  
    num = ii + 621 
    renamed_title = f"video_{num:04d}.mp4"
    highest_res_video = video.streams.filter(file_extension="mp4").order_by('resolution').last() 
    highest_res_video.download(output_path = directory, filename= f"video_{num:04d}.mp4")
    num += 1

    #dict of video info to store in our
    video_dict = {'title': video.title, 
                    'url': row['url'], 
                    'duration_seconds': video.length, 
                    'views': video.views, 
                    'relevant': row['relevant'],
                    'renamed_title': renamed_title}

    video_dict_list.append(video_dict)

    #pause slightly 
    time.sleep(0.5)

  except:
    print('Video {} at index {} unavailable...skipping'.format(row['url'], ii))
    #break
    
print('Done Downloading all videos at: {}'.format(directory))

Done Downloading all videos at: new_videos_to_add/


In [70]:
#create dataframe with new entries
print(len(video_dict_list))
hw_video_df = pd.DataFrame.from_dict(video_dict_list)
hw_video_df[-5:]


181


,title,url,duration_seconds,views,relevant,renamed_title
176,'This is Gross': Humpback Whale Carcass Liquid...,https://youtu.be/pXztGhYJXDI,76,6815,False,video_0813.mp4
177,RC Adventure Jeep Wrangler Axial SCX10 - The h...,https://youtu.be/YjY6PQqqMBg,298,256,False,video_0814.mp4
178,Top 10 small pigy animal in world,https://youtu.be/uKMJs4MGMEA,358,82,False,video_0815.mp4
179,Iceland: Top 10 Destination,https://youtu.be/xzBSPM2GppA,1080,76,False,video_0816.mp4
180,Top 10 Biggest Animal in the World,https://youtu.be/J_UF5cajbxo,322,82,False,video_0817.mp4


In [111]:
## create + save dataframe with downloaded yt mp4 video titles, relevant status, and duration etc
csv_file_name = 'humpback_whale_videos_with_intervals_additional.csv' #'humpback_whale_videos_with_intervals_additional_irrelevant.csv' #'humpback_whale_videos.csv'
hw_video_df.to_csv(csv_file_name)

In [91]:
# new_videos_to_add = pd.read_csv('new_videos_to_add.csv')
# new_videos_to_add.head(3)

,Unnamed: 0,title,url,relevant,occurence_intervals,double_checked,rand_sampled_intervals,double_checke
0,0,Humpback whale breach 7-10-14,https://youtu.be/I82N9I_SVmM,True,"[('0:30', '0:48')]",True,NaN,NaN
1,1,Whales-n-Sails flipper slapping humpback whale...,https://youtu.be/bE6FxjPbYcc,True,"[('0:36', '0:50')]",True,NaN,NaN
2,2,Whales-n-Sails humpback whales July 3/10,https://youtu.be/DIesiEj4XEw,True,"[('0:10', '0:17')]",True,NaN,NaN


In [98]:
# ## adding in occurence_intervals (+ other columns)
# # get list of renamed titles
# titles = list(hw_video_df['renamed_title'])

# #get list of the indices that the renamed titles match to in new_videos_to_add.csv
# indices = [int(name.split('video_')[1].split('.mp4')[0]) - 621 for name in titles]

# #get the occurence intervals as a list
# intervals = list(new_videos_to_add.loc[indices, ('occurence_intervals')])

# #add the occurence intervals as a column to 'humpback_whale_videos_with_intervals_additional.csv'
# #also add double_checked (True for all) and rand_sampled_intervals [None for all]
# hw_video_df['occurence_intervals'] = intervals
# hw_video_df['double_checked'] = [True for i in range(0, len(hw_video_df))]
# hw_video_df['rand_sampled_intervals'] = [None for i in range(0, len(hw_video_df))]

# Calculate Number of Frames for All Videos and Add Other Attributes to Dataframe of Downloaded Videos

In [112]:
#read csv back into google colab workspace
# upload = files.upload()

# csv_file_name = 'humpback_whale_videos_with_intervals_additional_irrelevant.csv' #'humpback_whale_videos.csv'
# hw_video_df = pd.read_csv(csv_file_name,index_col = 'Unnamed: 0')
hw_video_df.head()

,title,url,duration_seconds,views,relevant,renamed_title,occurence_intervals,double_checked,rand_sampled_intervals
0,Humpback whale breach 7-10-14,https://youtu.be/I82N9I_SVmM,59,63,True,video_0621.mp4,"[('0:30', '0:48')]",True,None
1,Whales-n-Sails flipper slapping humpback whale...,https://youtu.be/bE6FxjPbYcc,78,2123,True,video_0622.mp4,"[('0:36', '0:50')]",True,None
2,Whales-n-Sails humpback whales July 3/10,https://youtu.be/DIesiEj4XEw,52,84,True,video_0623.mp4,"[('0:10', '0:17')]",True,None
3,"Female Humpback Whale ""Sailing"" (Maui)",https://youtu.be/CnHl9Zr_07Y,145,1574,True,video_0624.mp4,"[('0:30', '0:45')]",True,None
4,"2015 Sardine Run 10, Humpback Whale",https://youtu.be/qjg8n42JpUk,19,37,True,video_0625.mp4,"[('0:00', '0:15')]",True,None


In [113]:
#upload FrameExtractor.py to google drive
# upload = files.upload()
from FrameExtractor import FrameExtractor

def get_num_frames(mp4_video):
  fe = FrameExtractor(mp4_video)

  #the number of frames in the video
  total_frames_available = fe.n_frames 

  return total_frames_available

In [114]:
#get renamed_titles (subselect sub_hw_df to find filtering index for each downloaded video)
# urls_downloaded_df = sub_hw_df[sub_hw_df['url'].isin(list(hw_video_df['url']))]
# renamed_titles = [f"video_{ii:04d}.mp4" for ii in urls_downloaded_df['filtering_index'] ]

#get duration_mins
duration_mins = list(hw_video_df['duration_seconds'] / 60)

# #occurence_intervals (None for each entry since all videos are irrelevant)
# occurence_intervals = [None for i in range(0, len(hw_video_df))]

# #double_checked --> true for each entry since we manually filtered to ensure videos were not relevant
# double_checked = [True for i in range(0, len(hw_video_df))]



In [116]:
## add in renamed_titles, duration_mins, occurence_intervals, double_checked
# hw_video_df['renamed_title'] = renamed_titles
hw_video_df['duration_mins'] = duration_mins
# hw_video_df['occurence_intervals'] = occurence_intervals
# hw_video_df['double_checked'] = double_checked
hw_video_df

,title,url,duration_seconds,views,relevant,renamed_title,occurence_intervals,double_checked,rand_sampled_intervals,duration_mins
0,Humpback whale breach 7-10-14,https://youtu.be/I82N9I_SVmM,59,63,True,video_0621.mp4,"[('0:30', '0:48')]",True,None,0.983333
1,Whales-n-Sails flipper slapping humpback whale...,https://youtu.be/bE6FxjPbYcc,78,2123,True,video_0622.mp4,"[('0:36', '0:50')]",True,None,1.300000
2,Whales-n-Sails humpback whales July 3/10,https://youtu.be/DIesiEj4XEw,52,84,True,video_0623.mp4,"[('0:10', '0:17')]",True,None,0.866667
3,"Female Humpback Whale ""Sailing"" (Maui)",https://youtu.be/CnHl9Zr_07Y,145,1574,True,video_0624.mp4,"[('0:30', '0:45')]",True,None,2.416667
4,"2015 Sardine Run 10, Humpback Whale",https://youtu.be/qjg8n42JpUk,19,37,True,video_0625.mp4,"[('0:00', '0:15')]",True,None,0.316667
...,...,...,...,...,...,...,...,...,...,...
176,'This is Gross': Humpback Whale Carcass Liquid...,https://youtu.be/pXztGhYJXDI,76,6815,False,video_0813.mp4,NaN,True,None,1.266667
177,RC Adventure Jeep Wrangler Axial SCX10 - The h...,https://youtu.be/YjY6PQqqMBg,298,256,False,video_0814.mp4,NaN,True,None,4.966667
178,Top 10 small pigy animal in world,https://youtu.be/uKMJs4MGMEA,358,82,False,video_0815.mp4,NaN,True,None,5.966667
179,Iceland: Top 10 Destination,https://youtu.be/xzBSPM2GppA,1080,76,False,video_0816.mp4,NaN,True,None,18.000000


In [117]:
# path = '/content//gdrive/MyDrive/'+ folder_name '/' #for gdrive
# path = 'additional_irrelevant_videos/' #for local use
path = 'new_videos_to_add/'


number_tot_frames = []

#get num total frames available for each video in our dataset
for title in hw_video_df['renamed_title']:
  video = path + title
  total_frames = get_num_frames(video)
  number_tot_frames.append(total_frames)

print(len(number_tot_frames))
number_tot_frames

181


[1773,
 2340,
 1576,
 4354,
 570,
 3503,
 2589,
 4422,
 1119,
 5182,
 14119,
 4767,
 7985,
 558,
 500,
 3457,
 1361,
 531,
 5280,
 2540,
 44029,
 2291,
 1167,
 4678,
 4405,
 10659,
 5496,
 2771,
 3064,
 2621,
 3095,
 936,
 1799,
 1305,
 258,
 1887,
 4956,
 3173,
 964,
 3942,
 6725,
 4085,
 4201,
 4035,
 2135,
 5509,
 7250,
 4996,
 1916,
 1951,
 2172,
 10667,
 3063,
 817,
 2615,
 803,
 4940,
 1542,
 3383,
 13380,
 2659,
 1949,
 1961,
 15880,
 17539,
 7479,
 561,
 2906,
 1489,
 4100,
 1182,
 5711,
 4929,
 2577,
 7148,
 820,
 4680,
 3019,
 5653,
 4255,
 2867,
 1414,
 5151,
 1366,
 4937,
 16021,
 18393,
 2320,
 4558,
 5335,
 1425,
 900,
 6104,
 4322,
 1800,
 3629,
 3007,
 4595,
 2617,
 2865,
 62342,
 4858,
 563,
 1790,
 10061,
 8526,
 1371,
 487,
 20036,
 5476,
 2216,
 5210,
 15800,
 7214,
 1390,
 4075,
 3742,
 3510,
 1377,
 4205,
 1768,
 2596,
 3719,
 39931,
 3755,
 8854,
 2749,
 557,
 16847,
 1251,
 5556,
 897,
 1414,
 4225,
 5280,
 2992,
 3323,
 1361,
 62828,
 3730,
 1929,
 4521,
 3980,

In [118]:
# add new column to df and redownload
hw_video_df['num_frames'] = number_tot_frames
hw_video_df

,title,url,duration_seconds,views,relevant,renamed_title,occurence_intervals,double_checked,rand_sampled_intervals,duration_mins,num_frames
0,Humpback whale breach 7-10-14,https://youtu.be/I82N9I_SVmM,59,63,True,video_0621.mp4,"[('0:30', '0:48')]",True,None,0.983333,1773
1,Whales-n-Sails flipper slapping humpback whale...,https://youtu.be/bE6FxjPbYcc,78,2123,True,video_0622.mp4,"[('0:36', '0:50')]",True,None,1.300000,2340
2,Whales-n-Sails humpback whales July 3/10,https://youtu.be/DIesiEj4XEw,52,84,True,video_0623.mp4,"[('0:10', '0:17')]",True,None,0.866667,1576
3,"Female Humpback Whale ""Sailing"" (Maui)",https://youtu.be/CnHl9Zr_07Y,145,1574,True,video_0624.mp4,"[('0:30', '0:45')]",True,None,2.416667,4354
4,"2015 Sardine Run 10, Humpback Whale",https://youtu.be/qjg8n42JpUk,19,37,True,video_0625.mp4,"[('0:00', '0:15')]",True,None,0.316667,570
...,...,...,...,...,...,...,...,...,...,...,...
176,'This is Gross': Humpback Whale Carcass Liquid...,https://youtu.be/pXztGhYJXDI,76,6815,False,video_0813.mp4,NaN,True,None,1.266667,2284
177,RC Adventure Jeep Wrangler Axial SCX10 - The h...,https://youtu.be/YjY6PQqqMBg,298,256,False,video_0814.mp4,NaN,True,None,4.966667,7459
178,Top 10 small pigy animal in world,https://youtu.be/uKMJs4MGMEA,358,82,False,video_0815.mp4,NaN,True,None,5.966667,8941
179,Iceland: Top 10 Destination,https://youtu.be/xzBSPM2GppA,1080,76,False,video_0816.mp4,NaN,True,None,18.000000,25887


In [119]:
# download updated dataframe
# csv_path= '/content/'+ csv_name #for gdrive use

csv_name = 'humpback_whale_videos_with_intervals_additional.csv'  #'humpback_whale_videos.csv'
hw_video_df.to_csv(csv_name)

# Finally, combine our new additional irrelevant videos csv with our humpback_whale_videos_with_intervals.csv

In [ ]:
#load in data
additional_irrelevant_df = pd.read_csv('humpback_whale_videos_with_intervals_additional_irrelevant.csv', index_col='Unnamed: 0')
intervals_df = pd.read_csv('humpback_whale_videos_with_intervals.csv', index_col='Unnamed: 0')

In [ ]:
additional_irrelevant_df.head()

,title,url,duration_seconds,views,relevant,renamed_title,duration_mins,occurence_intervals,double_checked,num_frames
0,Maui Adventure Tours Kayak Co.,https://youtu.be/MfvghiAzt70,36,63,False,video_0500.mp4,0.600000,NaN,True,977
1,Creen que esta es su finca privada | Flor Mizr...,https://youtu.be/OOrt72vZRc8,242,3639,False,video_0501.mp4,4.033333,NaN,True,7251
2,Japan's first commercial whale catch after 30 ...,https://youtu.be/ECMV6PcuBgs,123,247,False,video_0502.mp4,2.050000,NaN,True,3073
3,Cabo San Lucas: MARINA - Mexico (4K),https://youtu.be/BxH-fYBHggI,194,2797,False,video_0503.mp4,3.233333,NaN,True,4837
4,BEHIND THE SCENES | Futurismo Azores,https://youtu.be/jSX7_paIBpY,261,289,False,video_0504.mp4,4.350000,NaN,True,6255


In [ ]:
intervals_df.head()

,title,url,duration_seconds,views,relevant,renamed_title,duration_mins,num_frames,occurence_intervals,double_checked
0,Tom Mustill - Humpback Whales - A Detective St...,https://youtu.be/8j1HWD8KuMM,82,1909,True,video_0000.mp4,1.366667,2044,"[('0:00', '0:04'), ('0:30', '0:37'), ('1:11', ...",False
1,Suckling Humpback whale,https://youtu.be/tW_T8Z6ebmg,141,298156,True,video_0001.mp4,2.350000,0,"[('0:30', '0:40')]",False
2,Humpback Whales Lunge Feed Among Pod of Dolphi...,https://youtu.be/oSGhnD5kmAg,104,1117,True,video_0002.mp4,1.733333,3111,"[('0:00', '0:10'), ('0:42', '0:46')]",False
3,Humpback whales 2017 - Best of by Moorea Ocean...,https://youtu.be/6eWwFNq4i_c,224,2595,True,video_0003.mp4,3.733333,6719,"[('0:05', '0:20'), ('0:28', '0:33')]",False
4,Amazing! Humpback Whales Lunge Feeding up clos...,https://youtu.be/RMaMmynj76s,75,1005329,True,video_0004.mp4,1.250000,0,"[('0:00', '0:15'), ('0:35', '0:40')]",False


In [ ]:
#combine (concat) df's
frames = [intervals_df, additional_irrelevant_df]
final_dataset = pd.concat(frames, ignore_index=True)
final_dataset

,title,url,duration_seconds,views,relevant,renamed_title,duration_mins,num_frames,occurence_intervals,double_checked
0,Tom Mustill - Humpback Whales - A Detective St...,https://youtu.be/8j1HWD8KuMM,82,1909,True,video_0000.mp4,1.366667,2044,"[('0:00', '0:04'), ('0:30', '0:37'), ('1:11', ...",False
1,Suckling Humpback whale,https://youtu.be/tW_T8Z6ebmg,141,298156,True,video_0001.mp4,2.350000,0,"[('0:30', '0:40')]",False
2,Humpback Whales Lunge Feed Among Pod of Dolphi...,https://youtu.be/oSGhnD5kmAg,104,1117,True,video_0002.mp4,1.733333,3111,"[('0:00', '0:10'), ('0:42', '0:46')]",False
3,Humpback whales 2017 - Best of by Moorea Ocean...,https://youtu.be/6eWwFNq4i_c,224,2595,True,video_0003.mp4,3.733333,6719,"[('0:05', '0:20'), ('0:28', '0:33')]",False
4,Amazing! Humpback Whales Lunge Feeding up clos...,https://youtu.be/RMaMmynj76s,75,1005329,True,video_0004.mp4,1.250000,0,"[('0:00', '0:15'), ('0:35', '0:40')]",False
...,...,...,...,...,...,...,...,...,...,...
227,Bunker and Striped Bass off Fire Island,https://youtu.be/57JAgFhz144,349,5629,False,video_0622.mp4,5.816667,10469,NaN,True
228,Orcas Rubbing By,https://youtu.be/XnSAUMfpdU4,131,1957,False,video_0624.mp4,2.183333,3935,NaN,True
229,Deaths during the holiday season in Reykjavík,https://youtu.be/46SlPT-xidU,71,610,False,video_0625.mp4,1.183333,2138,NaN,True
230,humpback is unfair in deeeep.io,https://youtu.be/ha6nH_7xKBw,23,543,False,video_0627.mp4,0.383333,687,NaN,True


In [ ]:
final_dataset.drop(final_dataset.tail(6).index, inplace=True)
final_dataset

,title,url,duration_seconds,views,relevant,renamed_title,duration_mins,num_frames,occurence_intervals,double_checked
0,Tom Mustill - Humpback Whales - A Detective St...,https://youtu.be/8j1HWD8KuMM,82,1909,True,video_0000.mp4,1.366667,2044,"[('0:00', '0:04'), ('0:30', '0:37'), ('1:11', ...",False
1,Suckling Humpback whale,https://youtu.be/tW_T8Z6ebmg,141,298156,True,video_0001.mp4,2.350000,0,"[('0:30', '0:40')]",False
2,Humpback Whales Lunge Feed Among Pod of Dolphi...,https://youtu.be/oSGhnD5kmAg,104,1117,True,video_0002.mp4,1.733333,3111,"[('0:00', '0:10'), ('0:42', '0:46')]",False
3,Humpback whales 2017 - Best of by Moorea Ocean...,https://youtu.be/6eWwFNq4i_c,224,2595,True,video_0003.mp4,3.733333,6719,"[('0:05', '0:20'), ('0:28', '0:33')]",False
4,Amazing! Humpback Whales Lunge Feeding up clos...,https://youtu.be/RMaMmynj76s,75,1005329,True,video_0004.mp4,1.250000,0,"[('0:00', '0:15'), ('0:35', '0:40')]",False
...,...,...,...,...,...,...,...,...,...,...
221,5 Things You Need For Your Norway Winter Trip,https://youtu.be/A0oQYJUJWE4,11,130,False,video_0614.mp4,0.183333,280,NaN,True
222,Adolescent Grizzly Hunts Elk Herd || ViralHog,https://youtu.be/y3hSkTvFS-o,44,61298,False,video_0617.mp4,0.733333,1323,NaN,True
223,Lava Heat Causing Highway 132 Project Delays (...,https://youtu.be/-ogZNJy2IOY,451,6068,False,video_0618.mp4,7.516667,13525,NaN,True
224,DRIFTA FRASER ISLAND TRIP 2019 - COMING SOON,https://youtu.be/ASSf3MGJLsI,71,3527,False,video_0619.mp4,1.183333,1769,NaN,True


In [ ]:
print('Relevant: ', len(final_dataset[final_dataset['relevant'] == True]))
print('Not Relevant: ', len(final_dataset[final_dataset['relevant'] == False]))


Relevant:  113
Not Relevant:  113


In [ ]:
#save combined (+final) dataframe to the 'humpback_whale_videos_with_intervals.csv' file for consistency in our data preprocessing notebook
final_dataset.to_csv('humpback_whale_videos_with_intervals.csv')

# Miscellaneous

In [ ]:
# Instantiating GoogleDrive Service API to upload video files for storage efficiency
# https://www.youtube.com/watch?v=9K2P2bWEd90

CLIENT_SECRET_FILE = 'client_secret_file.json'
API_NAME = 'drive'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/drive']

service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)
# print(dir(service))

In [ ]:
## creating a google drive folder to upload our files to (the original folder with all irrel and rel items)
# folder_name = 'humpback_whale_yt_videos_capstone' #Folder ID: 1I6v1GfXk-RvRd3nGRRkysnJRW0LgTt0-
# folder_id = createFolder(folder_name, service)

# google drive folder to upload additional irrelevant videos
folder_name = 'humpback_whale_yt_videos_capstone_additional_irrelevant_videos' #Folder ID: 1RicLj-NgjnZDXsVGAqqokdsu1OlQb2vE
folder_id = createFolder(folder_name, service)
print('Folder name: {}\n Folder ID: {}'.format(folder_name, folder_id))

### Important Folder IDs (to avoid creating duplicate folders)
- Folder: 'humpback_whale_yt_videos_capstone' --> Folder ID: 1I6v1GfXk-RvRd3nGRRkysnJRW0LgTt0-
- Folder: 'humpback_whale_yt_videos_capstone_additional_irrelevant_videos' --> Folder ID: 1RicLj-NgjnZDXsVGAqqokdsu1OlQb2vE

In [ ]:
#upload a file onto google drive folder created
filename = 'Humpback Whales Lunge Feed Among Pod of Dolphins on San Diego Coast.mp4'
uploadFile(filename, service, folder_id)

In [ ]:
## upload video to google drive folder after getting mp4 object 
## instead of downloading to local file

highest_res_video.download(uploadFile(video.title +'.mp4', service, folder_id)) #lol it worked